<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://erachelson.github.io/RLclass_MVA/">https://erachelson.github.io/RLclass_MVA/</a>

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Chapter 5: Policy gradients</div>

So far, we have been mostly concerned with the problem of function approximation, that is "how do we store $q(s,a)$ in a convenient way, that is amenable to learning, retrieval and optimization?". A specific feature of the problems we tackled was that they had few discrete actions, making the dependency on $s$ the key difficulty. In particular, when looking for a $q$-greedy action, the $\max_{a\in A}$ problem had a straightforward solution as we could iterate through actions and retain the best one. We now turn to a more general case where the actions are too numerous to be enumerated (either because the action space is continuous or because it just has too many actions). Too many states motivated the introduction of function approximators for $v$ and $q$; too many actions similarly lead to function approximation for $\pi$. In the present chapter, we consider the general policy space (with possible approximation) and ask the question "how do we find a monotonically improving sequence of policies?".

<div class="alert alert-success">

**Learning outcomes**   
</div>

# Policy gradient methods

<div class="alert alert-success">

**Bottomline question:**   
The previous chapters have focussed on *action-value methods*; they aimed at estimating $Q^*$ in order to deduce $\pi^*$, or they jointly optimized $Q$ and $\pi$. Could we directly optimize $\pi$?
</div>

Suppose we have a policy $\pi_\theta$ parameterized by a vector $\theta$. Our goal is to find the parameter $\theta^*$ corresponding to $\pi^*$.

<div class="alert alert-warning">
    
**Exercise:**  
Recall the FrozenLake environment.  
How many states and how many actions were there in this environment? 
What would be a policy parameterization which does not make any approximation (ie. that can represent any policy in the policy space) for stationary, memoryless, stochastic policies?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

There are 16 states and 4 actions in the FrozenLake game.

A stationary, memoryless, stochastic policy is a mapping from $S$ to $\Delta_A$. Since $S$ and $A$ are discrete, it can be represented in tabular form as:
$$\pi = \left[ \begin{array}{cccc}
\pi(a_0|s_0) & \pi(a_1|s_0) & \pi(a_2|s_0) & 1 - \sum_{i=0}^2 \pi(a_i|s_0) \\
\ldots & & & \\
\pi(a_0|s_{|S|}) & \pi(a_1|s_{|S|}) & \pi(a_2|s_{|S|}) & 1 - \sum_{i=0}^2 \pi(a_i|s_{|S|})
\end{array} \right].$$

This parameterization enables representing any stochastic policy. It involves $|A|-1=3$ parameters per line, and $|S|=16$ lines, so in total $3\times 16 = 58$ parameters.

As previously, parameterization does not necessarily involve approximation! As the action set will become large or continuous, parameterization will enable generalization across actions at the cost of approximation, but a tabular representation is also a policy parameterization.
</details>

Remarks:
- $\pi_\theta$ might not be able to represent $\pi^*$. We will take a shortcut and call $\pi^*$ the best policy among the $\pi_\theta$ ones.
- For discrete state and action spaces, the tabular policy representation is a special case of policy parameterization.
- Policy parameterization is a (possibly useful) way of introducing prior knowledge on the set of the desired policies.
- The optimal deterministic policies might not belong to the policy subspace of $\pi_\theta$, thus it makes sense to consider stochastic policies for $\pi_\theta$.
- It makes even more sense to consider stochastic policies that it opens the family of environments that we can tackle, like partially observable MDPs or multi-player games.

For stochastic policies, we shall write $\pi_\theta(a|s)$.

In the remainder of the chapter, we will assume that $\pi_\theta(a|s)$ is differentiable with respect to $\theta$.

Suppose now we define some performance metric $J(\pi_\theta) = J(\theta)$. If $J$ is differentiable and a stochastic estimate $\tilde{\nabla}_\theta J(\theta)$ of the gradient is available, then we can define the stochastic gradient ascent update procedure:
$$\theta \leftarrow \theta + \alpha \tilde{\nabla}_\theta J(\theta).$$

We will call **policy gradient methods** all methods that follow such a procedure (whether or not they also learn a value function or not).

<div class="alert alert-success">

**Policy gradient method**   
We call **policy gradient method** any method that performs stochastic gradient ascent on the policy's parameters.  
Given a stochastic estimate $\tilde{\nabla}_\theta J(\theta)$ of a policy's performance criterion with respect to the policy's parameters, such a method implements the update procedure: 
$$\theta \leftarrow \theta + \alpha \tilde{\nabla}_\theta J(\theta).$$
</div>

Remarks: 
- Note that $J$ is a generic criterion. For example, $J$ could be defined as the $\gamma$-discounted value of a starting state (or a distribution of starting states), or as the undiscounted reward over a certain horizon, or as the average reward.
- Note that this family of methods can use any gradient estimate for $\tilde{\nabla}_\theta J(\theta)$: formal calculus, finite differences, automated differentiation, evolution strategies, etc.
- Why is it interesting to look at methods which explicitly store a policy function? Because the evaluation of the policy in a given state $s$ does not require the maximization step ($\max_a Q(s,a)$), which might be computationally costly, especially for continuous actions. Instead, it replaces it with a call to $\pi_\theta(s)$ (or a draw from $\pi_\theta(a|s)$). This argument makes actor-critic architectures or direct policy search a method of choice for continuous actions domains (especially common in Robotics) and Policy Gradient is one of them.
- When do policy gradient approaches outperform value-based ones? It's hard to give a precise criterion; it really depends on the problem. One thing that comes into play is how easy it is to approximate the optimal policy or the optimal value function. If one is simpler than the other (by "simpler", we mean "it is easier to find a parameterization whose spanned function space almost includes the function to approximate"), then it is a good heuristic to try to approximate it. But this criterion might itself be hard to assess.

<div class="alert alert-warning">

**Exercise**  
From the class on Markov Decision Processes, can you recall a scalar criterion $J(\pi)$ whose optimization is provably equivalent to finding a policy that dominates any other one in every state?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>

Provided $\rho_0$ has non-zero probability mass on all states, an optimal policy is a solution to $\max_\pi J(\pi) = \mathbb{E}_{s_0\sim \rho_0}[V^\pi(s_0)]$.

This makes optimizing $J(\pi)$ a legitimate goal for finding optimal policies: from now on we will work with $J(\pi)$.
</details>

This very short chapter serves as a header to two families of estimators for policy gradients, which we will study separately.   

First, we will look at **Monte Carlo policy gradient** methods. These methods rely on simulating one or several full trajectories with $\pi$ to compute $\nabla J$. They are quite intuitive and make for a good first contact with the problem of estimating $\nabla J$. These methods include the very well-known REINFORCE method and its modern offsprings, but also most of the literature on evolutionary reinforcement learning.  

Then, we will follow the same kind of reasoning the enabled moving from $v(s) = \sum_t \gamma^t r_t$ to the Bellman equation, and will study how $\nabla J$ can be expressed as a quantity one can estimate from independent $(s,a,r,s')$ samples. All the corresponding algorithms will be motivated by the famous **policy gradient theorem** which will enable an interesting connection with what we have already covered in approximate dynamic programming.